In [60]:
import numpy as np
from scipy.stats import binom, multinomial
import streamlit as st

In [115]:
def single_roll(n, thresh):
    '''
    calculates the distribution of a single roll with n dice, with everything >= thresh being a sucess and every 6 being a crit
    the resulting matrix has the form
    [p_00 p_01 p_02...
     p_10 p_11 p_12...
     ... 
    ]
    where p_ij is the probability of i hits and j crits
    (where of course p_ij = 0 if i+j>n)
    '''
    results = np.zeros((n+1,n+1))
    p_crit = 1/6
    p_hit = (6-thresh)/6
    p = [p_hit,p_crit,1-p_hit-p_crit]
    for i in range(n+1):
        for j in range(n+1-i):
            results[i,j] = multinomial.pmf([i,j,n-i-j], n=n, p=p)
    return results

def roll(distr, thresh):
    '''
    distr is a list e.g. [0.25,0.5,0.25] meaning
    25% chance of n=0
    50% chance of n=1
    25% chance of n=2
    For each possible n-value we calculate the probability distribution of a roll with n dice, padding the result to a uniform size and taking the weighted average of all of them
    using the values from distr as weights
    '''
    max_n = len(distr)
    resulting_distr = np.zeros((max_n,max_n))
    for n, prob in enumerate(distr):
        n_distr = single_roll(n,thresh)
        n_distr = np.pad(n_distr,(0,max_n-n-1), mode="constant", constant_values=0)
        resulting_distr += prob * n_distr
    return resulting_distr


def get_amount_of_hits(distr):
    '''
    calculates a 1-d distribution of the amount of successes given the matrix of hits and crits
    here hits and crits are treated the same, so no extra rules implemented yet
    '''
    n = distr.shape[0]
    resulting_distr = [0]*n
    for i in range(n):
        for j in range(i+1):
            resulting_distr[i] += distr[j,i-j]
    return resulting_distr



In [116]:
test = roll([0.25,0.25,0.5],4)
get_amount_of_hits(test)

[0.5, 0.375, 0.125]